In [1]:
import torch
import os
import time

# For first run only to set proper directory
os.chdir('..')

import pandas as pd
import numpy as np

from torch.utils.data.dataloader import DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
from emotions_model_training.ERNN import ERNN
from emotions_model_training.dl_ops import dl_ops
from SQL.connector import connector
from emotions_model_training.EmotionsDataset import EmotionsDataset

In [2]:
# Paramteres for scripts (implement proportion of sets)
sets_proportion = [0.6, 0.2, 0.2]       # train / valid / test
batch = 64                              # batch size
do_shuffle = True                       # shuffling data in loader
learn_rate = 1e-3                       # learning rate for training
num_epoch = 1                           # training epochs

In [3]:
# Loading the dataset with SQL query and dispaying for insight
db_dir = "D:\\Repos\\Face_Detection_PyTorch\\dataset\\expression_dataset\\data\\data\\image\\origin\\"

conn = connector()
db = conn.query(f'SELECT * FROM dbo.label')

display(db)


d:\Repos\Face_Detection_PyTorch\SQL\connector.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, connection)


,image,face_id_image,box_top,box_left,box_right,box_bottom,box_confidence,label
0,distaste_mother_319.jpg,3,120,664,712,168,33.1615,6
1,distaste_mother_498.jpg,1,175,103,150,222,7.75454,3
2,distaste_mother_58.jpg,0,72,88,136,120,72.9418,6
3,distaste_mother_623.jpg,0,254,5,72,321,79.3727,6
4,distaste_mother_667.jpg,0,67,180,315,202,61.9211,1
...,...,...,...,...,...,...,...,...
91788,surprised_expression_546.jpg,0,70,70,351,351,37.7117,5
91789,surprised_expression_381.jpg,0,51,61,117,107,91.6307,5
91790,surprised_expression_395.jpg,0,27,95,258,190,96.2861,5
91791,ecstatic_asian_31.jpg,0,60,136,184,108,39.9223,3


In [4]:
# Splitting data into sets (train, test, valid) - with renumeration
train, val_test = train_test_split(db, test_size=0.4, train_size=0.6, random_state=42, stratify=db['label'])
valid, test = train_test_split(val_test, test_size=0.5, train_size=0.5, random_state=42, stratify=val_test['label'])

train, valid, test = train.reset_index(drop=True), valid.reset_index(drop=True), test.reset_index(drop=True)

print("Checking sizes of prepared datasets:")
print("Train shape: " + str(train.shape[0]) + " (" + str(round((train.shape[0]/db.shape[0])*100)) + "%)")
print("Valid shape: " + str(valid.shape[0]) + " (" + str(round((valid.shape[0]/db.shape[0])*100)) + "%)")
print("Test shape: " + str(test.shape[0]) + " (" + str(round((test.shape[0]/db.shape[0])*100)) + "%)")

Checking sizes of prepared datasets:
Train shape: 55075 (60%)
Valid shape: 18359 (20%)
Test shape: 18359 (20%)


In [5]:
# Init for datasets with PyTorch class
train_dataset = EmotionsDataset(train, db_dir)

# Fixing valid and test dataset to same sizes (for convinience with NN architecture)
horz, vert = train_dataset.horz_max, train_dataset.vert_max

valid_dataset = EmotionsDataset(valid, db_dir, horz, vert)
test_dataset = EmotionsDataset(test, db_dir, horz, vert)

# Init for dataloaders with default batch = 64
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch, shuffle=do_shuffle, drop_last=True)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=batch, shuffle=do_shuffle, drop_last=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch, shuffle=do_shuffle, drop_last=True)


In [6]:
# Checking maximal size - in theory it can vary, but it depends from split of dataset
print("Size of images: " + str(horz) + " x " + str(vert))

Size of images: 2117 x 2117


In [7]:
# Checking CUDA avalibility
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

print("Using " + str(device))


Using cuda


In [8]:
# Defining model and parameters
ernn = ERNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adagrad(ernn.parameters(), lr=learn_rate)

# Creating operational class for proceding with train, valid, test process
dl_ops = dl_ops(model=ernn, loss_fn=loss_fn, optimizer=optim)
print(f"Architecture of current ERNN: \n {ernn}")

Architecture of current ERNN: 
 ERNN(
  (conv1): Conv2d(3, 1, kernel_size=(197, 197), stride=(120, 120))
  (relu1): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (lin1): Linear(in_features=289, out_features=7, bias=True)
  (soft): Softmax(dim=None)
)


In [9]:
print(dl_ops.train(train_dataloader, epoch=num_epoch, device=device))

d:\Repos\Face_Detection_PyTorch\face_venv\Lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


KeyboardInterrupt: 